In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd() / "_scripts"))

In [3]:
import polars as pl
from great_tables import GT
from great_tables.data import gtcars
import altair as alt

from collections import namedtuple, defaultdict
from itertools import product
from TexSoup import TexSoup

In [40]:
from plan2eplus.helpers.variable_interfaces import all_variables as vars
from plan2eplus.studies.analysis2.all_cases_v_time import create_space_and_site_dfs
from plan2eplus.studies.experiments.name_splits import (
    split_by_case_type,
    split_by_case_type_and_alias,
    split_by_doors,
)
from plan2eplus.studies.experiments.retrieve import (
    retrieve_comparison_groups,
)
from plan2eplus.studies.experiments.scatter import (
    create_comparison_df,
    create_plots_for_comparison_group,
)
from plan2eplus.studies.setup.data_wrangle2 import (
    create_wide_dataframe_for_many_qois_and_cases,
)

from plan2eplus.helpers.helpers import chain_flatten, ContainsAsEqualsString

In [5]:
cases = retrieve_comparison_groups("doors")
cases

# get_experiment_folders()

[CaseData(case_name='bol_5_CLOSED', idf=<geomeppy.idf.IDF object at 0x14acdea50>, sql=Energy SQLiteResult: /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/plan2eplus/cases/241119_doors/bol_5_CLOSED/results/eplusout.sql, path_to_input=PosixPath('/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/svg2plan/outputs2/case_bol_5'), path_to_output=PosixPath('/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/plan2eplus/cases/241119_doors/bol_5_CLOSED')),
 CaseData(case_name='amb_b1_DYNAMIC', idf=<geomeppy.idf.IDF object at 0x14facb4d0>, sql=Energy SQLiteResult: /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/plan2eplus/cases/241119_doors/amb_b1_DYNAMIC/results/eplusout.sql, path_to_input=PosixPath('/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/svg2plan/outputs2/case_amb_b1'), path_to_output=PosixPath('/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/plan2eplus/cases/241119_doors/amb_b1_DYNAMIC')),
 CaseData(case_name='red_b1_DYNAMIC', idf=<geomeppy.idf.IDF object at 0x14fac8910>, sql=Energy SQLiteResult

In [6]:
df_temp = create_wide_dataframe_for_many_qois_and_cases(
    cases, [vars.zone.temp["zone_mean_air_temp"]]
)

df, df_site = create_space_and_site_dfs(
    cases=cases,
    space_qois=[vars.afn.zone["vent_vol"], vars.afn.zone["mix_vol"]],
    site_qois=[vars.site.wind["speed"]],
)

In [7]:
df_vol = df.join(df_temp, on=["case_names", "space_names", "datetimes"]).with_columns(
    combined_vent=pl.col("AFN Zone Ventilation Volume [m3]")
    + pl.col("AFN Zone Mixing Volume [m3]")
)
df_vol.head()

case_names,space_names,datetimes,AFN Zone Ventilation Volume [m3],AFN Zone Mixing Volume [m3],time,Zone Mean Air Temperature [C],combined_vent
str,str,datetime[μs],f64,f64,str,f64,f64
"""bol_5_CLOSED""","""BLOCK 00 STOREY 0""",2017-07-01 00:00:00,240.133294,0.0,"""00:00""",19.624067,240.133294
"""bol_5_CLOSED""","""BLOCK 00 STOREY 0""",2017-07-01 00:15:00,281.920251,0.0,"""00:15""",18.62186,281.920251
"""bol_5_CLOSED""","""BLOCK 00 STOREY 0""",2017-07-01 00:30:00,308.423544,0.0,"""00:30""",17.635697,308.423544
"""bol_5_CLOSED""","""BLOCK 00 STOREY 0""",2017-07-01 00:45:00,324.762766,0.0,"""00:45""",16.662282,324.762766
"""bol_5_CLOSED""","""BLOCK 00 STOREY 0""",2017-07-01 01:00:00,309.539315,0.0,"""01:00""",16.507866,309.539315


In [8]:
df_agg = df_vol.group_by("case_names").agg(
    [
        pl.mean("AFN Zone Ventilation Volume [m3]"),
        pl.mean("AFN Zone Mixing Volume [m3]"),
        pl.mean("combined_vent"),
        pl.mean("Zone Mean Air Temperature [C]"),
    ]
)

In [29]:
df_agg = split_by_doors(split_by_case_type_and_alias(df_agg))
df_agg

case_names,AFN Zone Ventilation Volume [m3],AFN Zone Mixing Volume [m3],combined_vent,Zone Mean Air Temperature [C],case_type,exp_type
str,f64,f64,f64,f64,str,str
"""red_b1_CLOSED""",232.640805,146.860422,379.501227,22.327406,"""C""","""CLOSED"""
"""amb_b1_CLOSED""",70.374518,208.903753,279.278271,23.553939,"""A""","""CLOSED"""
"""amb_b1_DYNAMIC""",85.99218,291.524465,377.516645,23.418397,"""A""","""DYNAMIC"""
"""red_b1_Medium""",299.684163,405.075236,704.759398,22.126302,"""C""","""OPEN"""
"""bol_5_CLOSED""",317.663785,92.129051,409.792836,23.154855,"""B""","""CLOSED"""
"""bol_5_DYNAMIC""",326.89749,144.66714,471.564629,23.126356,"""B""","""DYNAMIC"""
"""amb_b1_Medium""",91.36269,353.926941,445.28963,23.111333,"""A""","""OPEN"""
"""bol_5_Medium""",340.079384,250.772877,590.852261,23.040847,"""B""","""OPEN"""
"""red_b1_DYNAMIC""",262.983758,253.302264,516.286022,22.288867,"""C""","""DYNAMIC"""


### charts

In [44]:
alt.Chart(df_agg).mark_point().encode(
    color="exp_type",
)

alt.Chart(...)

### tables

In [11]:
qois = ["vent", "mix", "temp"]
stat_names = ["mean", "std", "max_diff"]
other_cols = ["_".join(i) for i in list(product(qois, stat_names))]
Stats = namedtuple("Stats", ["study_type", *other_cols])
Stats


def create_stats(filter_exp: pl.Expr, exp_name: str):
    qois = [
        "AFN Zone Ventilation Volume [m3]",
        "AFN Zone Mixing Volume [m3]",
        "Zone Mean Air Temperature [C]",
    ]

    def get_stats(qoi):
        case = df_agg.filter(filter_exp)[qoi]
        return [case.mean(), case.std(), case.max() - case.min()]

    all_stats = chain_flatten([get_stats(qoi) for qoi in qois])
    return Stats(exp_name, *all_stats)

In [31]:
res = [
    create_stats(pl.col("exp_type") == "OPEN", "Across")._asdict(),
    create_stats(pl.col("case_type") == "A", "A")._asdict(),
    create_stats(pl.col("case_type") == "B", "B")._asdict(),
    create_stats(pl.col("case_type") == "C", "C")._asdict(),
]

dd = defaultdict(list)

for d in res:  # you can list as many input dicts as you want here
    for key, value in d.items():
        dd[key].append(value)
df_stats = pl.DataFrame(dd)
df_stats

study_type,vent_mean,vent_std,vent_max_diff,mix_mean,mix_std,mix_max_diff,temp_mean,temp_std,temp_max_diff
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Across""",243.708746,133.472597,248.716695,336.591684,78.598268,154.302359,22.759494,0.549491,0.985031
"""A""",82.576463,10.903035,20.988171,284.785053,72.746106,145.023187,23.361223,0.226774,0.442606
"""B""",328.213553,11.265602,22.4156,162.523023,80.815161,158.643826,23.107353,0.059332,0.114008
"""C""",265.102908,33.571879,67.043358,268.412641,129.768891,258.214813,22.247525,0.106736,0.201103


In [41]:




def col_fix(i:str) -> str:
    cstring = ContainsAsEqualsString(i)
    match cstring:
        case "mean":
            return "Mean"
        case "std":
            return "Std."
        case "max_diff":
            return "Max. Diff."
        case _:
            return i
        

In [46]:
# def col_fix(i):
#     return i
        
tbl = (
    GT(df_stats)
    .tab_header(title="Door Experiment")
    # .tab_stub(rowname_col="study_type")
    .tab_spanner(
        label="Ventilation Volume [m3/s]",
        columns=other_cols[0:3]
    )
    .tab_spanner(
        label="Mixing Volume [m3/s]",
        columns=other_cols[3:6],
    )
    .tab_spanner(label="Temperature [ºC]", columns=other_cols[6:])
    .fmt_number(columns=pl.selectors.numeric(), n_sigfig=2, )  #or decimals.. 
    .cols_label({i: col_fix(i) for i in other_cols})

)
tbl

GT(_tbl_data=shape: (4, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ study_typ ┆ vent_mean ┆ vent_std  ┆ vent_max_ ┆ … ┆ mix_max_d ┆ temp_mean ┆ temp_std ┆ temp_max_ │
│ e         ┆ ---       ┆ ---       ┆ diff      ┆   ┆ iff       ┆ ---       ┆ ---      ┆ diff      │
│ ---       ┆ f64       ┆ f64       ┆ ---       ┆   ┆ ---       ┆ f64       ┆ f64      ┆ ---       │
│ str       ┆           ┆           ┆ f64       ┆   ┆ f64       ┆           ┆          ┆ f64       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ Across    ┆ 243.70874 ┆ 133.47259 ┆ 248.71669 ┆ … ┆ 154.30235 ┆ 22.759494 ┆ 0.549491 ┆ 0.985031  │
│           ┆ 6         ┆ 7         ┆ 5         ┆   ┆ 9         ┆           ┆          ┆           │
│ A         ┆ 82.576463 ┆ 10.903035 ┆ 20.988171 ┆ … ┆ 145.02318 ┆ 23.361223 ┆ 0.226774 ┆ 0.442606  │
│           ┆           ┆           ┆           ┆   ┆ 7         ┆           ┆          ┆           │
│ B         ┆ 328.21355 ┆ 11.265602 ┆ 22.4156   ┆ … ┆ 158.64382 ┆ 23.107353 ┆ 0.059332 ┆ 0.114008  │
│           ┆ 3         ┆           ┆           ┆   ┆ 6         ┆           ┆          ┆           │
│ C         ┆ 265.10290 ┆ 33.571879 ┆ 67.043358 ┆ … ┆ 258.21481 ┆ 22.247525 ┆ 0.106736 ┆ 0.201103  │
│           ┆ 8         ┆           ┆           ┆   ┆ 3         ┆           ┆          ┆           │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴──────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x15a6edb30>, _boxhead=Boxhead([ColInfo(var='study_type', type=<ColInfoTypeEnum.default: 1>, column_label='study_type', column_align='left', column_width=None), ColInfo(var='vent_mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='right', column_width=None), ColInfo(var='vent_std', type=<ColInfoTypeEnum.default: 1>, column_label='Std.', column_align='right', column_width=None), ColInfo(var='vent_max_diff', type=<ColInfoTypeEnum.default: 1>, column_label='Max. Diff.', column_align='right', column_width=None), ColInfo(var='mix_mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='right', column_width=None), ColInfo(var='mix_std', type=<ColInfoTypeEnum.default: 1>, column_label='Std.', column_align='right', column_width=None), ColInfo(var='mix_max_diff', type=<ColInfoTypeEnum.default: 1>, column_label='Max. Diff.', column_align='right', column_width=None), ColInfo(var='temp_mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='right', column_width=None), ColInfo(var='temp_std', type=<ColInfoTypeEnum.default: 1>, column_label='Std.', column_align='right', column_width=None), ColInfo(var='temp_max_diff', type=<ColInfoTypeEnum.default: 1>, column_label='Max. Diff.', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x14d1890d0>, _spanners=Spanners([SpannerInfo(spanner_id='Ventilation Volume [m3/s]', spanner_level=0, spanner_label='Ventilation Volume [m3/s]', spanner_units=None, spanner_pattern=None, vars=['vent_mean', 'vent_std', 'vent_max_diff'], built=None), SpannerInfo(spanner_id='Mixing Volume [m3/s]', spanner_level=0, spanner_label='Mixing Volume [m3/s]', spanner_units=None, spanner_pattern=None, vars=['mix_mean', 'mix_std', 'mix_max_diff'], built=None), SpannerInfo(spanner_id='Temperature [ºC]', spanner_level=0, spanner_label='Temperature [ºC]', spanner_units=None, spanner_pattern=None, vars=['temp_mean', 'temp_std', 'temp_max_diff'], built=None)]), _heading=Heading(title='Door Experiment', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x14d189550>, _formats=[<great_tables._gt_data.FormatInfo object at 0x14d188ad0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, cat

In [15]:
print(tbl.as_latex())

\begin{table}[!t]
\caption*{
{\large Door Experiment}
} 

\fontsize{12.0pt}{14.4pt}\selectfont

\begin{tabular*}{\linewidth}{@{\extracolsep{\fill}}lrrrrrrrrr}
\toprule
 & \multicolumn{3}{c}{Vent} & \multicolumn{3}{c}{Mix} & \multicolumn{3}{c}{Temp} \\ 
\cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10}
study\_type & Mean & Std & Max Diff & Mean & Std & Max Diff & Mean & Std & Max Diff \\ 
\midrule\addlinespace[2.5pt]
Across Plans & 240 & 130 & 250 & 340 & 79 & 150 & 23 & 0.55 & 0.99 \\
Red & 270 & 34 & 67 & 270 & 130 & 260 & 22 & 0.11 & 0.20 \\
Amber & 83 & 11 & 21 & 280 & 73 & 150 & 23 & 0.23 & 0.44 \\
Bol & 330 & 11 & 22 & 160 & 81 & 160 & 23 & 0.059 & 0.11 \\
\bottomrule
\end{tabular*}

\end{table}



In [16]:
vent = (
    alt.Chart(df_agg)
    .mark_point()
    .encode(
        x=alt.X("case_type:N"),
        y=alt.Y(r"AFN Zone Ventilation Volume \[m3\]:Q"),
        color="exp_type",
    )
)